In [1]:
import mlflow
from mlflow.tracking import MlflowClient

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

client = MlflowClient()

# Use the search_experiments function to get a list of available experiments
experiments = client.search_experiments()
print(len(experiments))

# Display the list of available experiments
for experiment in experiments:
    print(f"Experiment Name: {experiment.name}, Experiment ID: {experiment.experiment_id}")

tracking URI: 'file:///Users/viviane/Desktop/MLOps/NYC-home-value/mlruns'
5
Experiment Name: nyc_house_price_linear, Experiment ID: 296524759276928674
Experiment Name: nyc_house_price_ensemble, Experiment ID: 624000600208382101
Experiment Name: nyc_house_price_xgb, Experiment ID: 879898173824708793
Experiment Name: nyc_house_price_randomforest, Experiment ID: 686346702712954572
Experiment Name: Default, Experiment ID: 0


In [48]:
import pandas as pd
def load_data(path):
    return pd.read_csv(path)

df = load_data('data/nyc-rolling-sales.csv')
df.head()
# df.shape

,NO:,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,19-07-2017 00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,14-12-2016 00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,09-12-2016 00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,23-09-2016 00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,17-11-2016 00:00


In [49]:
from datetime import datetime

def clean_data(df):
    df = df.dropna()
    df['SALE DATE'] = pd.to_datetime(df['SALE DATE'], format="%d-%m-%Y %H:%M")
    df['house_age'] = df['SALE DATE'].dt.year - df['YEAR BUILT']
    df = df[(df['ZIP CODE']!=0) & (df['YEAR BUILT']!=0)]
    df = df[['NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS AT PRESENT', 'ZIP CODE','TOTAL UNITS', 'GROSS SQUARE FEET', 'house_age', 'SALE PRICE']]
    df['SALE PRICE'] = pd.to_numeric(df['SALE PRICE'], errors='coerce')
    df['GROSS SQUARE FEET'] = pd.to_numeric(df['GROSS SQUARE FEET'], errors='coerce')
    df.rename(columns={'BUILDING CLASS CATEGORY': 'building_category',
                    'BUILDING CLASS AT PRESENT': 'building_class',
                    'ZIP CODE': 'zip_code',
                    'TOTAL UNITS': 'total_unit',
                    'GROSS SQUARE FEET': 'square_feet',
                    'SALE PRICE': 'price'
                    }, inplace=True)
    df['building_category'] = df['building_category'].str[:3].apply(lambda x: x.strip())
    columns_to_replace = ['total_unit', 'square_feet', 'house_age']
    mean_values = df[columns_to_replace].mean()
    df[columns_to_replace] = df[columns_to_replace].replace(0, mean_values)
    df[columns_to_replace] = df[columns_to_replace].fillna(mean_values)
    df = df[(df['price'] >=20000) & (df['price'] <=3000000)]
    return df

df = clean_data(df)

In [11]:
# df.describe()
df.head()

,NEIGHBORHOOD,building_category,building_class,zip_code,total_unit,square_feet,house_age,price
13,ALPHABET CITY,09,C6,10009,2.394866,4256.917964,97.0,499000.0
15,ALPHABET CITY,09,C6,10009,2.394866,4256.917964,97.0,529500.0
16,ALPHABET CITY,09,C6,10009,2.394866,4256.917964,97.0,423000.0
17,ALPHABET CITY,09,C6,10009,2.394866,4256.917964,92.0,501000.0
18,ALPHABET CITY,09,C6,10009,2.394866,4256.917964,96.0,450000.0


In [ ]:
# Import libraries
# !pip install plotly
# !pip install nbformat==4.2.0


import seaborn as sns
import matplotlib.pyplot as plt
import re
import plotly.express as px

fig = px.box(df, y='price',color_discrete_sequence=px.colors.sequential.Agsunset,
             width=600, height=500)
fig.update_layout(title_text='Box Plot of Price')
fig.show()

In [49]:
# df.describe()
df.nunique()
# df.isna().sum()

NEIGHBORHOOD          235
building_category      40
building_class        119
zip_code              167
square_feet          3475
year_built            143
price                4986
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
CATEGORICAL_COLS = ["NEIGHBORHOOD", "building_category", "building_class"]
NUMERICAL_COLS = ["zip_code", "total_unit", "square_feet", "house_age"]

In [107]:
from typing import List
from sklearn.feature_extraction import DictVectorizer

def encode_cols(df: pd.DataFrame, categorical_cols: List[str] = None, numerical_cols: List[str] = None) -> pd.DataFrame:
    if categorical_cols is None:
        categorical_cols = ["NEIGHBORHOOD", "building_category", "building_class"]
    if numerical_cols is None:
        numerical_cols = ["zip_code", "total_unit", "square_feet", "house_age"]

    df[numerical_cols] = df[numerical_cols].fillna(-1).astype("float")
    df[categorical_cols] = df[categorical_cols].apply(lambda x: x.astype(str).str.lower())
    return df


def extract_x_y(
    df: pd.DataFrame,
    categorical_cols: List[str] = None,
    numerical_cols: List[str] = None,
    dv: DictVectorizer = None,
    with_target: bool = True,
) -> dict:
    if categorical_cols is None:
        categorical_cols = ["NEIGHBORHOOD", "building_category", "building_class"]
    if numerical_cols is None:
        numerical_cols = ["zip_code", "total_unit", "square_feet", "house_age"]
    dicts = df[[*categorical_cols, *numerical_cols]].to_dict(orient="records")

    y = None
    if with_target:
        if dv is None:
            dv = DictVectorizer()
            dv.fit(dicts)
        y = df["price"].values

    x = dv.transform(dicts)
    return x, y, dv

# save the preprocessor into saved_pkl folder
import pickle
def save_picked(path: str, file):
    with open(path, "wb") as f:
        pickle.dump(file, f)

In [101]:
# Train model
# !pip install xgboost

import numpy as np
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def train_model(x_train: pd.DataFrame, y_train: np.ndarray, model_type):
    # model = None
    random = RandomForestRegressor(random_state=42, n_estimators=25,
                              max_depth=60, min_samples_leaf=1, min_samples_split=5)
    xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.25, n_estimators=300,
                                max_depth=6, subsample=1, colsample_bytree=1)
    linear = LinearRegression(fit_intercept=True, copy_X=True)

    model_map = {
        "randomforest": random,
        "xgb": xgb_reg,
        "linear": linear
    }
    model = model_map.get(model_type, None)
    if model is None:
        raise ValueError(f"Invalid model type: {model_type}")
    
    model.fit(x_train, y_train)
    return model

def predict_price(input_data, model):
    return model.predict(input_data)

def evaluate_model(y_true: np.ndarray, y_pred: np.ndarray):
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    return rmse, mae, r2

In [56]:
# try all steps
df = load_data('/Users/viviane/Desktop/MLOps/NYC-home-value/data/nyc-rolling-sales.csv')
df = clean_data(df)
df.to_csv("/Users/viviane/Desktop/MLOps/NYC-home-value/data/nyc-house-price-cleaned.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = encode_cols(train_df)
test_df = encode_cols(test_df)
X_train, y_train, dv = extract_x_y(train_df)
X_test, y_test, _ = extract_x_y(test_df, dv=dv)

In [102]:
model = train_model(X_train, y_train, "randomforest")
pred = predict_price(X_test, model)
rmse, mae, r2 = evaluate_model(y_test, pred)
print(f"r2: {r2}")


r2: 0.5694


In [103]:
model = train_model(X_train, y_train, "xgb")
pred = predict_price(X_test, model)
rmse, mae, r2 = evaluate_model(y_test, pred)
print(f"r2: {r2}")


r2: 0.5795


In [59]:
model = train_model(X_train, y_train, "linear")
pred = predict_price(X_test, model)
rmse, mae, r2 = evaluate_model(y_test, pred)
print(f"r2: {r2}")


r2: 0.359


In [104]:
# run MLflow
import mlflow.sklearn
# from lib.config import PATH_TO_PREPROCESSOR

# input model type: xgb/linear/randomforest
model_type = "xgb"
mlflow_experiment_path = f"nyc_house_price_{model_type}"

# Set the experiment name
mlflow.set_experiment(mlflow_experiment_path)
data_path = 'data/nyc-rolling-sales.csv'

# Start a run
with mlflow.start_run() as run:
    run_id = run.info.run_id

    # Set tags for the run
    mlflow.set_tag("stage", "training")

    # Load data
    df = load_data(data_path)
    df = clean_data(df)
    df.to_csv('data/nyc-house-price-cleaned.csv')

    # Split train, test dataset
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Encode categorical and numerical columns
    train_df = encode_cols(train_df)

    # Extract X and y
    X_train, y_train, dv = extract_x_y(train_df)

    # Save preprocessor
    save_picked("/Users/viviane/Desktop/MLOps/NYC-home-value/web_service/saved_pkl/dv__v0.0.1.pkl", dv)

    # Train model, log model
    model = train_model(X_train, y_train, model_type)
    mlflow.sklearn.log_model(model, "model")

    # Define params, log params
    params = model.get_params()
    mlflow.log_params(params)

    # Evaluate model
    prediction = predict_price(X_train, model)
    train_mse, train_mae, train_r2 = evaluate_model(y_train, prediction)

    # Log metrics
    mlflow.log_metric("Train-MSE", train_mse)
    mlflow.log_metric("Train-MAE", train_mae)
    mlflow.log_metric("Train-R2", train_r2)

    # Evaluate model on test set
    test_df = encode_cols(test_df)
    X_test, y_test, _ = extract_x_y(test_df, dv=dv)
    y_pred_test = predict_price(X_test, model)
    test_mse, test_mae, test_r2 = evaluate_model(y_test, y_pred_test)

    # Log metrics
    mlflow.log_metric("Test-MSE", test_mse)
    mlflow.log_metric("Test-MAE", test_mae)
    mlflow.log_metric("Test-R2", test_r2)

    # Register your model in mlfow model registry
    registered_model = mlflow.register_model(f"runs:/{run_id}/model", mlflow_experiment_path)

Registered model 'nyc_house_price_xgb' already exists. Creating a new version of this model...
2024/01/28 17:29:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc_house_price_xgb, version 5
Created version '5' of model 'nyc_house_price_xgb'.


In [105]:
from mlflow.tracking import MlflowClient
client = MlflowClient()

model_type = "xgb"
mlflow_experiment_path = f"nyc_house_price_{model_type}"

production_version = 5
client.transition_model_version_stage(name=mlflow_experiment_path, version=production_version, stage="Production")

<ModelVersion: aliases=[], creation_timestamp=1706459374452, current_stage='Production', description=None, last_updated_timestamp=1706459377447, name='nyc_house_price_xgb', run_id='fba069075db14f92bd271aba05074265', run_link=None, source='file:///Users/viviane/Desktop/MLOps/NYC-home-value/mlruns/578894006709759806/fba069075db14f92bd271aba05074265/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [19]:
!mlflow ui --host 0.0.0.0 --port 5002

[2024-01-27 23:19:17 +0100] [48214] [INFO] Starting gunicorn 21.2.0
[2024-01-27 23:19:17 +0100] [48214] [INFO] Listening at: http://0.0.0.0:5002 (48214)
[2024-01-27 23:19:17 +0100] [48214] [INFO] Using worker: sync
[2024-01-27 23:19:17 +0100] [48215] [INFO] Booting worker with pid: 48215
[2024-01-27 23:19:17 +0100] [48216] [INFO] Booting worker with pid: 48216
[2024-01-27 23:19:17 +0100] [48217] [INFO] Booting worker with pid: 48217
[2024-01-27 23:19:17 +0100] [48218] [INFO] Booting worker with pid: 48218
^C
[2024-01-27 23:19:48 +0100] [48214] [INFO] Handling signal: int
[2024-01-27 23:19:48 +0100] [48217] [INFO] Worker exiting (pid: 48217)
[2024-01-27 23:19:48 +0100] [48218] [INFO] Worker exiting (pid: 48218)
[2024-01-27 23:19:48 +0100] [48215] [INFO] Worker exiting (pid: 48215)
[2024-01-27 23:19:48 +0100] [48216] [INFO] Worker exiting (pid: 48216)


In [111]:
# from config import PATH_TO_MODEL, PATH_TO_PREPROCESSOR
# Load production model
model_uri = f"models:/{mlflow_experiment_path}/production"
model = mlflow.sklearn.load_model(model_uri)
save_picked("/Users/viviane/Desktop/MLOps/NYC-home-value/web_service/saved_pkl/model__v0.0.1.pkl", model)

def load_pickle(path):
    with open(path, "rb") as f:
        file = pickle.load(f)
    return file

dv = load_pickle("/Users/viviane/Desktop/MLOps/NYC-home-value/web_service/saved_pkl/dv__v0.0.1.pkl")
model = load_pickle("/Users/viviane/Desktop/MLOps/NYC-home-value/web_service/saved_pkl/model__v0.0.1.pkl")

In [110]:
# Try predicting house price
features = ['NEIGHBORHOOD', 'building_category', 'building_class', 'zip_code','total_unit', 'square_feet', 'house_age']
user_input = {}

for f in features:
    user_input[f] = input(f"Enter value for {f}: ")

# Print input values
print("\nInput Values:")
for feature, value in user_input.items():
    print(f"{feature}: {value}")

X_pred = pd.DataFrame([user_input])
X_pred = encode_cols(X_pred)
dicts = X_pred.to_dict(orient="records")
X_pred = dv.transform(dicts)
y_pred = predict_price(X_pred, model)

print(f"Predicted house price: {round(y_pred[0]):,.0f} USD")


Input Values:
NEIGHBORHOOD: CHELSEA
building_category: 09
building_class: R3
zip_code: 10011
total_unit: 3
square_feet: 2000
house_age: 14
Predicted house price: 1,812,121 USD
